In [1]:
import os
from tarefa02.gcrf import Gcrf, Point, Edge, Face

In [2]:
gc = Gcrf()

In [3]:
p = Point(1, 2, 3)
print(p)

(1, 2, 3)


In [4]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)

e = Edge(p1, p2)

print(e)

[(1, 2, 3) -> (4, 5, 6)]


In [5]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)
p3 = Point(7, 8, 9)

e1 = Edge(p1, p2)
e2 = Edge(p2, p3)
e3 = Edge(p3, p1)

f = Face(e1, e2, e3)
f_ = Face(e3, e1, e2)
print(f)

{[(1, 2, 3) -> (4, 5, 6)], [(4, 5, 6) -> (7, 8, 9)], [(7, 8, 9) -> (1, 2, 3)]}


In [6]:
def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False

In [7]:
p1 = Point(0,-1,0)
p2 = Point(0, 0, 0)
p3 = Point(0,0,1)
a = Point(1, 0,0)
b = Point(-1,0,0)
print(gc.signed_volume(p1,p2,p3,a))
print(gc.signed_volume(p1,p2,p3,b))

0.16666666666666666
-0.16666666666666666


In [8]:
def PivotAroundEdge(e, P):
#     print()
#     print("Entrada:", e)
    p = P[-1]
    area2 = gc.square_area(e.p1, e.p2, p)
    for p_ in range(len(P)-1):
        volume = gc.signed_volume(e.p1, e.p2, p, P[p_])
#         print("P",p)
#         print("P_", P[p_])
#         print("VOLUME", volume)
        if volume > 0:
            p = P[p_]
        elif volume == 0:
            area2_ = gc.square_area(e.p1, e.p2, P[p_])
#             print(area2)
#             print(area2_)
            if area2_ > area2:
                p = P[p_]
                area2 = area2_
#     print("Saída:", p)
#     print()
    return p

In [9]:
def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

In [10]:
def FindEdgeOnHull(P):
    p = BottomMostLeftMostBackMost(P)
    q = p

    for r in P:
        if (q.z == r.z) and (q.y == r.y) and (q.x < r.x):
            q = r
    
    if (q == p):
        q = Point(p.x + 1, p.y, p.z)
#         print("Chamando PIVOT no Find Edge on Hull")
        q = PivotAroundEdge(Edge(p, q), P)

    return Edge(p, q)

In [11]:
def FindTriangleOnHull(P):
    e = FindEdgeOnHull(P)
    r = PivotAroundEdge(e, P)
    e2 = Edge(e.p2, r)
    e3 = Edge(r, e.p1)
    f = Face(e, e2, e3)
    return f

In [12]:
def NotProcessed(E, e):
#     print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True

In [13]:
def GiftWrap(P):
    t = FindTriangleOnHull(P)
    #print("Triangulo encontrado:", t)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = []

    while not len(Q) == 0:
        e = Q.pop()
        e = Edge(e.p2, e.p1)
        #print("Aresta a ser processada:", e)
        if NotProcessed(Processed, e):
            #print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(e, P)
            #print("Resultado do PivotAroundEdge:", q)
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
            #print("Nova face:", t)
            face_no_hull = checa_face(H, t)
            if not face_no_hull:
                H.append(t)
                #print("{} entrou no Hull".format(t))
#             e_ = Edge(e.p2, e.p1)
            Q.extend([t.e1, t.e2, t.e3])
            #print("Q agora possui as arestas:")
            #for q in Q: print(q)
            Processed.append(e)
            #print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
            #for aresta in Processed: print(aresta)
        #else: print("Aresta JÁ PROCESSADA")
    return H

In [14]:
# def GiftWrap(P):
#     t = FindTriangleOnHull(P)
#     print("Triangulo encontrado:", t)
#     Q = [t.e1, t.e2, t.e3]
#     H = [t]
#     Processed = []
#     while not len(Q) == 0:
#         e = Q.pop()
#         print("Aresta a ser processada:", e)
#         if NotProcessed(Processed, e):
#             print()
#             print("Aresta {} não foi processada, entrando no IF".format(e))
#             q = PivotAroundEdge(e, P)
#             print("Resultado do PivotAroundEdge:", q)
#             print()
#             t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
#             print("Nova face:", t)
#             face_no_hull = checa_face(H, t)
#             if not face_no_hull:
#                 H.append(t)
#                 print("{} entrou no Hull".format(t))
#             e_ = Edge(e.p2, e.p1)
#             Q.extend([e_, t.e2, t.e3])
#             print("Q agora possui as arestas:")
#             for q in Q: print(q)
#             Processed.append(e)
#             print("Processadas:")
#             for p in Processed: print(p)
# #             print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
# #             for aresta in Processed: print(aresta)
#         else: print("Aresta JÁ PROCESSADA")
#     return H

In [15]:
# p1 = Point(-3, 0, 0)
# p2 = Point(0, -3, 0)
# p3 = Point(0, 2, 0)
# p4 = Point(0, 0, 3)
# p5 = Point(0, 0, -4)
# p6 = Point(4, 0, 0)
# p7 = Point(0, 0, 0)

p1 = Point(-1, 0, 0)
p2 = Point(0, -1, 0)
p3 = Point(0, 1, 0)
p4 = Point(0, 0, 1)
p5 = Point(0, 0, -1)
p6 = Point(1, 0, 0)
p7 = Point(0, 0, 0)

P = [p1, p2, p3, p4, p5, p6, p7]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))

{[(0, -1, 0) -> (0, 0, 1)], [(0, 0, 1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)]}
{[(0, -1, 0) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, 0, -1)], [(0, 0, -1) -> (0, -1, 0)]}
{[(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (1, 0, 0)], [(1, 0, 0) -> (0, -1, 0)]}
{[(0, -1, 0) -> (1, 0, 0)], [(1, 0, 0) -> (0, 0, 1)], [(0, 0, 1) -> (0, -1, 0)]}
{[(-1, 0, 0) -> (0, 0, 1)], [(0, 0, 1) -> (0, 1, 0)], [(0, 1, 0) -> (-1, 0, 0)]}
{[(-1, 0, 0) -> (0, 1, 0)], [(0, 1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (-1, 0, 0)]}
{[(0, 0, -1) -> (0, 1, 0)], [(0, 1, 0) -> (1, 0, 0)], [(1, 0, 0) -> (0, 0, -1)]}
{[(1, 0, 0) -> (0, 1, 0)], [(0, 1, 0) -> (0, 0, 1)], [(0, 0, 1) -> (1, 0, 0)]}
8


In [16]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                    i1 = j
                if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                    i2 = j
                if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


In [17]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                i1 = j
            if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                i2 = j
            if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                i3 = j
                
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [18]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
    return points

In [19]:
points = read_vertices_from_obj(r"C:\Users\romul\Documents\UFC\MESTRADO\Disciplinas\Geometria Computacional\super-nintendo-blend\p1.obj")
print(len(points))

348


In [20]:
nH = GiftWrap(points)
for h in nH:
    print(h)
print(len(nH))

{[(-0.258873, 0.032391, 0.264508) -> (-0.25596, 0.032736, 0.268785)], [(-0.25596, 0.032736, 0.268785) -> (-0.266031, 0.037396, 0.275269)], [(-0.266031, 0.037396, 0.275269) -> (-0.258873, 0.032391, 0.264508)]}
{[(-0.258873, 0.032391, 0.264508) -> (-0.266031, 0.037396, 0.275269)], [(-0.266031, 0.037396, 0.275269) -> (-0.356125, 0.037052, 0.135614)], [(-0.356125, 0.037052, 0.135614) -> (-0.258873, 0.032391, 0.264508)]}
{[(-0.258873, 0.032391, 0.264508) -> (-0.356125, 0.037052, 0.135614)], [(-0.356125, 0.037052, 0.135614) -> (-0.346055, 0.032391, 0.129129)], [(-0.346055, 0.032391, 0.129129) -> (-0.258873, 0.032391, 0.264508)]}
{[(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)], [(-0.346055, 0.032391, 0.129129) -> (-0.340916, 0.033424, 0.136366)], [(-0.340916, 0.033424, 0.136366) -> (-0.258873, 0.032391, 0.264508)]}
{[(-0.258873, 0.032391, 0.264508) -> (-0.340916, 0.033424, 0.136366)], [(-0.340916, 0.033424, 0.136366) -> (-0.265298, 0.034456, 0.253046)], [(-0.265298, 0.03

{[(-0.377771, 0.060465, 0.085175) -> (-0.369577, 0.042379, 0.138125)], [(-0.369577, 0.042379, 0.138125) -> (-0.378144, 0.063908, 0.08212)], [(-0.378144, 0.063908, 0.08212) -> (-0.377771, 0.060465, 0.085175)]}
{[(-0.378144, 0.063908, 0.08212) -> (-0.369577, 0.042379, 0.138125)], [(-0.369577, 0.042379, 0.138125) -> (-0.378331, 0.067006, 0.079604)], [(-0.378331, 0.067006, 0.079604) -> (-0.378144, 0.063908, 0.08212)]}
{[(-0.378331, 0.067006, 0.079604) -> (-0.369577, 0.042379, 0.138125)], [(-0.369577, 0.042379, 0.138125) -> (-0.378336, 0.070966, 0.07675)], [(-0.378336, 0.070966, 0.07675) -> (-0.378331, 0.067006, 0.079604)]}
{[(-0.378336, 0.070966, 0.07675) -> (-0.369577, 0.042379, 0.138125)], [(-0.369577, 0.042379, 0.138125) -> (-0.377859, 0.075786, 0.074027)], [(-0.377859, 0.075786, 0.074027) -> (-0.378336, 0.070966, 0.07675)]}
{[(-0.377859, 0.075786, 0.074027) -> (-0.369577, 0.042379, 0.138125)], [(-0.369577, 0.042379, 0.138125) -> (-0.377332, 0.079229, 0.072369)], [(-0.377332, 0.079229, 

In [21]:
print_faces(H, P)
print_faces(nH, points)
write_faces(H, P)

f 2 4 1
f 2 1 5
f 2 5 6
f 2 6 4
f 1 4 3
f 1 3 5
f 5 3 6
f 6 3 4
f 279 278 170
f 279 170 181
f 279 181 296
f 279 296 294
f 279 294 282
f 279 282 268
f 279 268 336
f 279 336 341
f 279 341 344
f 279 344 347
f 347 344 345
f 347 345 346
f 347 346 339
f 347 339 345
f 345 339 346
f 345 346 335
f 345 335 340
f 345 340 346
f 346 340 337
f 346 337 339
f 339 337 341
f 339 341 344
f 339 344 347
f 344 341 343
f 344 343 345
f 345 343 340
f 340 343 342
f 340 342 344
f 340 344 347
f 340 347 346
f 344 342 343
f 343 342 345
f 345 342 346
f 346 342 347
f 347 342 344
f 343 341 342
f 342 341 338
f 342 338 345
f 345 338 346
f 346 338 347
f 347 338 344
f 344 338 341
f 341 338 336
f 341 336 343
f 343 336 342
f 342 336 340
f 340 336 337
f 337 336 338
f 337 338 339
f 339 338 341
f 338 336 306
f 338 306 337
f 337 306 308
f 337 308 339
f 339 308 310
f 339 310 314
f 339 314 322
f 339 322 346
f 346 322 335
f 335 322 341
f 335 341 336
f 335 336 340
f 341 322 338
f 338 322 337
f 337 322 339
f 322 314 318
f 322 318 34

f 345 242 340
f 340 242 336
f 336 242 295
f 295 242 297
f 295 297 296
f 297 242 332
f 332 242 333
f 333 242 335
f 335 242 249
f 249 242 246
f 249 246 248
f 249 248 130
f 249 130 129
f 249 129 250
f 249 250 252
f 249 252 345
f 345 252 340
f 340 252 336
f 336 252 332
f 332 252 333
f 333 252 335
f 335 252 322
f 322 252 282
f 282 252 336
f 282 336 338
f 282 338 337
f 282 337 308
f 282 308 310
f 282 310 314
f 282 314 322
f 252 250 336
f 336 250 295
f 295 250 332
f 332 250 333
f 333 250 335
f 335 250 322
f 322 250 346
f 346 250 345
f 345 250 340
f 340 250 336
f 250 129 252
f 252 129 200
f 252 200 253
f 252 253 345
f 345 253 340
f 340 253 336
f 336 253 295
f 295 253 332
f 332 253 333
f 333 253 335
f 335 253 255
f 335 255 322
f 322 255 282
f 282 255 263
f 282 263 268
f 282 268 336
f 336 268 332
f 332 268 333
f 333 268 335
f 335 268 322
f 322 268 346
f 346 268 345
f 345 268 340
f 340 268 336
f 268 263 336
f 336 263 332
f 332 263 333
f 333 263 335
f 335 263 322
f 322 263 305
f 305 263 346
f 346 

f 318 180 319
f 319 180 320
f 320 180 321
f 321 180 322
f 322 180 323
f 323 180 324
f 324 180 325
f 325 180 327
f 327 180 328
f 328 180 329
f 329 180 331
f 331 180 332
f 332 180 333
f 333 180 334
f 334 180 335
f 335 180 336
f 336 180 295
f 295 180 296
f 296 180 297
f 297 180 298
f 298 180 299
f 299 180 300
f 300 180 301
f 301 180 303
f 303 180 304
f 304 180 305
f 305 180 306
f 306 180 307
f 307 180 308
f 308 180 309
f 309 180 310
f 310 180 311
f 311 180 312
f 312 180 313
f 180 230 228
f 180 228 225
f 180 225 318
f 318 225 319
f 319 225 320
f 230 181 277
f 230 277 276
f 230 276 335
f 230 335 334
f 230 334 333
f 230 333 337
f 230 337 336
f 230 336 335
f 277 181 278
f 278 181 279
f 181 169 335
f 181 335 337
f 181 337 336
f 181 336 340
f 181 340 342
f 181 342 345
f 181 345 344
f 181 344 347
f 181 347 346
f 335 169 243
f 335 243 244
f 243 169 244
f 244 169 245
f 245 169 246
f 246 169 248
f 248 169 249
f 249 169 250
f 250 169 252
f 252 169 253
f 253 169 254
f 254 169 255
f 255 169 256
f 256 

f 250 131 252
f 252 131 253
f 253 131 254
f 254 131 255
f 255 131 256
f 256 131 257
f 257 131 259
f 259 131 260
f 260 131 261
f 261 131 262
f 262 131 263
f 263 131 264
f 264 131 265
f 265 131 266
f 131 82 231
f 231 82 213
f 213 82 166
f 166 82 165
f 165 82 124
f 165 124 271
f 271 124 123
f 123 124 333
f 123 333 337
f 123 337 336
f 123 336 340
f 123 340 342
f 123 342 343
f 123 343 345
f 123 345 344
f 123 344 341
f 123 341 342
f 333 124 334
f 334 124 335
f 335 124 237
f 237 124 235
f 235 124 236
f 236 124 238
f 238 124 239
f 239 124 240
f 240 124 241
f 241 124 242
f 242 124 243
f 243 124 244
f 244 124 245
f 245 124 246
f 246 124 248
f 248 124 249
f 249 124 250
f 250 124 252
f 252 124 253
f 253 124 254
f 254 124 255
f 255 124 256
f 256 124 257
f 257 124 259
f 259 124 260
f 260 124 261
f 261 124 262
f 262 124 263
f 263 124 264
f 264 124 265
f 265 124 266
f 266 124 267
f 267 124 268
f 268 124 269
f 269 124 270
f 270 124 271
f 124 82 123
f 123 82 104
f 123 104 128
f 123 128 274
f 128 104 127

f 140 108 139
f 139 108 232
f 232 108 221
f 221 108 171
f 171 108 130
f 130 108 200
f 130 200 250
f 130 250 335
f 130 335 334
f 130 334 337
f 130 337 340
f 130 340 342
f 130 342 343
f 130 343 345
f 130 345 346
f 130 346 347
f 200 108 136
f 136 108 164
f 164 108 220
f 220 108 199
f 199 108 198
f 198 108 168
f 168 108 167
f 167 108 218
f 218 108 217
f 217 108 194
f 194 108 132
f 132 108 131
f 131 108 231
f 231 108 213
f 213 108 166
f 166 108 165
f 108 85 107
f 107 85 103
f 103 85 91
f 103 91 174
f 103 174 239
f 103 239 160
f 103 160 187
f 103 187 157
f 103 157 158
f 103 158 190
f 103 190 191
f 103 191 209
f 103 209 183
f 103 183 184
f 174 91 173
f 173 91 197
f 197 91 140
f 140 91 139
f 139 91 232
f 232 91 221
f 221 91 171
f 171 91 130
f 130 91 129
f 129 91 136
f 136 91 135
f 135 91 164
f 164 91 220
f 220 91 134
f 134 91 133
f 133 91 198
f 198 91 168
f 168 91 167
f 167 91 218
f 218 91 217
f 217 91 194
f 194 91 132
f 132 91 131
f 131 91 231
f 231 91 213
f 213 91 166
f 166 91 165
f 165 91 1

f 167 34 218
f 218 34 217
f 217 34 194
f 194 34 132
f 132 34 131
f 131 34 231
f 231 34 213
f 213 34 166
f 166 34 165
f 165 34 124
f 124 34 123
f 123 34 128
f 128 34 127
f 127 34 161
f 161 34 216
f 216 34 170
f 170 34 169
f 169 34 181
f 181 34 223
f 223 34 233
f 233 34 144
f 144 34 143
f 143 34 204
f 204 34 226
f 226 34 177
f 177 34 304
f 34 32 33
f 34 33 208
f 208 33 207
f 207 33 32
f 207 32 31
f 31 32 103
f 31 103 209
f 31 209 183
f 31 183 184
f 31 184 228
f 31 228 234
f 31 234 316
f 31 316 315
f 31 315 337
f 31 337 336
f 31 336 340
f 31 340 342
f 31 342 343
f 31 343 345
f 103 32 36
f 32 33 59
f 32 59 107
f 32 107 53
f 32 53 103
f 107 59 103
f 103 59 69
f 59 33 233
f 59 233 296
f 59 296 279
f 59 279 216
f 59 216 231
f 59 231 131
f 59 131 194
f 59 194 217
f 59 217 218
f 59 218 167
f 59 167 198
f 59 198 199
f 59 199 133
f 59 133 220
f 59 220 232
f 59 232 139
f 59 139 140
f 59 140 197
f 59 197 173
f 59 173 174
f 59 174 187
f 59 187 157
f 59 157 158
f 59 158 190
f 59 190 209
f 59 209 228


f 221 101 171
f 171 101 130
f 130 101 200
f 200 101 136
f 136 101 135
f 135 101 164
f 164 101 220
f 220 101 134
f 134 101 133
f 133 101 199
f 199 101 198
f 198 101 168
f 168 101 167
f 167 101 218
f 218 101 217
f 217 101 108
f 108 101 103
f 103 101 159
f 159 101 239
f 101 100 173
f 173 100 197
f 197 100 140
f 140 100 139
f 139 100 232
f 232 100 221
f 221 100 171
f 171 100 130
f 130 100 200
f 200 100 136
f 136 100 135
f 135 100 164
f 164 100 220
f 220 100 134
f 134 100 133
f 133 100 199
f 199 100 198
f 198 100 168
f 168 100 167
f 167 100 218
f 218 100 217
f 217 100 194
f 194 100 132
f 132 100 231
f 231 100 213
f 213 100 166
f 166 100 165
f 165 100 193
f 193 100 128
f 128 100 127
f 127 100 161
f 161 100 216
f 216 100 170
f 170 100 169
f 169 100 181
f 181 100 223
f 223 100 233
f 233 100 144
f 144 100 143
f 143 100 204
f 204 100 226
f 226 100 177
f 177 100 176
f 176 100 208
f 208 100 207
f 207 100 146
f 146 100 145
f 145 100 186
f 186 100 185
f 185 100 224
f 224 100 180
f 180 100 154
f 154 

In [22]:
write_faces(nH, points)